# Gastebois Jacques
## Phase 0 : Generation des Trajectoires Expertes (Buffers)

Avant de distiller, il nous faut des "maitres" a copier. On entraine ici plusieurs reseaux (des ConvNets par defaut) sur le vrai dataset et on enregistre leurs poids a chaque etape.

C'est un peu comme enregistrer les differentes prises d'un musicien chez Funky Junk pour pouvoir les analyser plus tard :)

---

### 1. on prepare le terrain

import des modules de base. on verifie bien qu'on a le mps ou la cuda si on est sur une grosse machine.

In [ ]:
import os
import torch
import torch.nn as  nn
import numpy as  np
from tqdm import tqdm
import copy

from utils import get_dataset, get_network, get_daparam, TensorDataset, epoch, ParamDiffAug

if torch.cuda.is_available():
    dev = 'cuda'
elif torch.backends.mps.is_available():
    dev = 'mps'
else:
    dev = 'cpu'
    
print(f"ok on va bosser sur {dev} !!")

### 2. reglages pour les experts

on regle combien d'experts on veut et pendant combien de temps ils vont s'entrainer. le papier dit 100 experts sur 50 epochs pour cifar10. c'est long mais c'est le prix de la qualite :p

In [ ]:
class buffer_config:
    dataset = 'CIFAR10'
    model = 'ConvNet'
    num_experts = 200
    epochs = 50
    lr_teacher = 0.01
    mom = 0.9
    l2 = 5e-4
    batch_train = 256
    zca = True 
    data_path = './data'
    save_path = './buffers_experts'

args =  buffer_config()
args.device = dev
args.dsa_param = ParamDiffAug()
args.dsa_strategy = 'color_crop_cutout_flip_scale_rotate'

if  not os.path.exists(args.save_path):
    os.makedirs(args.save_path)
    print(f"dossier {args.save_path} cree :)")

### 3. chargement du dataset reel

on recupere les vraies images pour entrainer nos experts. 

In [ ]:
print("chargement des donnees...")
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader, loader_train_dict, class_map, class_map_inv = get_dataset(args.dataset, args.data_path, args.batch_train, '', args=args)

print(f"on a {len(dst_train)} images d'entrainement. c'est parti :)")

### 4. gros oeuvre : entrainement des experts

pour chaque expert, on l'entraine et on garde une trace de ses parametres a chaque epoch. 
on range ca dans des fichiers .pt pour la suite. 

In [ ]:
def generate_experts():
    experts_trajectories = []
    
    for i  in range(args.num_experts):
        print(f"\\nentrainement de l'expert n° {i+1}/{args.num_experts}")
        
        net = get_network(args.model, channel, num_classes, im_size).to(dev)
        net.train()
        
        optim = torch.optim.SGD(net.parameters(), lr=args.lr_teacher, momentum=args.mom, weight_decay=args.l2)
        criterion = nn.CrossEntropyLoss().to(dev)
        
        trajectoire = []
        # on sauve l'etat initial
        trajectoire.append([p.detach().cpu().clone() for p in  net.parameters()])
        
        for e in range(args.epochs):
            # une epoch d'entrainement standard
            loss, acc = epoch('train', loader_train_dict, net, optim, criterion, args, aug=True)
            
            # on sauve les poids apres l'epoch
            trajectoire.append([p.detach().cpu().clone() for p in net.parameters()])
            
            if (e+1) % 10 == 0:
                print(f"   epoch {e+1}/{args.epochs} - acc: {acc:.2%}")
        
        experts_trajectories.append(trajectoire)
        
        # on sauve regulierement pour pas tout perdre si ca plante
        if (i+1) % 1 == 0:
            save_name = os.path.join(args.save_path, f"replay_buffer_{i}.pt")
            # on sauve juste cet expert dans son fichier
            torch.save([trajectoire], save_name)
            print(f"expert {i} sauve dans {save_name} :)")


generate_experts()